In [1]:
import xarray as xr
import numpy as np
import sys
sys.path.append('/home/563/rl5183/PhD-research/Functions')
import functions as func
from importlib import reload

In [2]:
#read in obs and detrend 
ersst_ds = xr.open_dataset('/g/data/eg3/jxb548/OBSDATA/ersstv5.1950_2018.nc')
ersst_sst = ersst_ds.sst
#detrend
ersst_detrend = np.apply_along_axis(func.detrend_2step, 0, ersst_sst)
ersst_sst_dt = xr.DataArray(ersst_detrend, coords=ersst_sst.coords, attrs=ersst_sst.attrs)

In [3]:
#calculate nino34 index
nino34_index = func.nino34_index(ersst_sst_dt, '1950-01', '1979-12')

In [4]:
#calculate el nino and la nina years using the DJF nino34 index
yearly_seasonal_nino34_index = func.seasonal_mean_nino_index(nino34_index)
yearly_seasonal_nino34_index_djf = yearly_seasonal_nino34_index.sel(season='DJF').drop_sel(seasonyear=2019)
el_nino_years_djf = yearly_seasonal_nino34_index_djf.where(yearly_seasonal_nino34_index_djf>0.4, drop=True).seasonyear
la_nina_years_djf = yearly_seasonal_nino34_index_djf.where(yearly_seasonal_nino34_index_djf<-0.4, drop=True).seasonyear
el_nino_years_djf = el_nino_years_djf -1
la_nina_years_djf = la_nina_years_djf -1

In [5]:
#calculate el nino and la nina years using 6-month nino34 index
el_nino_years_six_month, la_nina_years_six_month = func.find_event_years(nino34_index, 0.4, 6)

In [6]:
el_nino_years_six_month

<xarray.DataArray 'year' (time: 20)>
array([1951, 1953, 1957, 1963, 1965, 1968, 1972, 1976, 1977, 1982, 1986,
       1991, 1993, 1994, 1997, 2002, 2004, 2006, 2009, 2014])
Coordinates:
  * time        (time) datetime64[ns] 1951-07-01 1953-04-01 ... 2014-12-01
    month       (time) int64 7 4 5 7 7 10 7 10 10 6 10 5 5 7 7 7 8 10 9 12
    seasonyear  (time) int64 1951 1953 1957 1963 1965 ... 2004 2006 2009 2015

In [7]:
el_nino_years_djf

<xarray.DataArray 'seasonyear' (seasonyear: 26)>
array([1951, 1953, 1957, 1958, 1963, 1965, 1968, 1969, 1972, 1976, 1977,
       1979, 1982, 1986, 1987, 1990, 1991, 1994, 1997, 2002, 2003, 2004,
       2006, 2009, 2014, 2015])
Coordinates:
    season      <U3 'DJF'
  * seasonyear  (seasonyear) int64 1952 1954 1958 1959 ... 2007 2010 2015 2016

In [8]:
la_nina_years_six_month

<xarray.DataArray 'year' (time: 13)>
array([1950, 1954, 1964, 1970, 1973, 1984, 1988, 1995, 1998, 2007, 2010,
       2016, 2017])
Coordinates:
  * time        (time) datetime64[ns] 1950-05-01 1954-08-01 ... 2017-12-01
    month       (time) int64 5 8 7 9 7 12 7 11 9 9 8 10 12
    seasonyear  (time) int64 1950 1954 1964 1970 1973 ... 2007 2010 2016 2018

In [9]:
la_nina_years_djf

<xarray.DataArray 'seasonyear' (seasonyear: 25)>
array([1950, 1954, 1955, 1956, 1962, 1964, 1970, 1971, 1973, 1974, 1975,
       1983, 1984, 1988, 1995, 1998, 1999, 2000, 2005, 2007, 2008, 2010,
       2011, 2016, 2017])
Coordinates:
    season      <U3 'DJF'
  * seasonyear  (seasonyear) int64 1951 1955 1956 1957 ... 2011 2012 2017 2018

### Find event initiation month and duration

In [10]:
import climtas

In [11]:
el_nino_events = climtas.event.find_events(nino34_index >= 0.4, min_duration=6)

In [12]:
el_nino_events

,time,event_duration
0,18,11
1,39,12
2,88,17
3,162,10
4,186,13
5,225,19
6,270,11
7,321,8
8,333,6
9,389,15


In [13]:
times_en=el_nino_events.time.values

In [14]:
initiation_months_en = nino34_index.isel(time=times_en).month

In [15]:
initiation_months_en

<xarray.DataArray 'month' (time: 20)>
array([ 7,  4,  5,  7,  7, 10,  7, 10, 10,  6, 10,  5,  5,  7,  7,  7,  8,
       10,  9, 12])
Coordinates:
  * time        (time) datetime64[ns] 1951-07-01 1953-04-01 ... 2014-12-01
    month       (time) int64 7 4 5 7 7 10 7 10 10 6 10 5 5 7 7 7 8 10 9 12
    seasonyear  (time) int64 1951 1953 1957 1963 1965 ... 2004 2006 2009 2015

In [16]:
duration_en = el_nino_events.event_duration.values

In [17]:
duration_en

array([11, 12, 17, 10, 13, 19, 11,  8,  6, 15, 19, 17,  6, 11, 13, 10, 10,
        6,  9, 20])

In [18]:
la_nina_events = climtas.event.find_events(nino34_index <= -0.4, min_duration=6)

In [19]:
la_nina_events

,time,event_duration
0,4,12
1,55,30
2,174,9
3,248,19
4,282,36
5,419,11
6,462,14
7,550,7
8,584,33
9,692,12


In [20]:
times_ln = la_nina_events.time.values

In [21]:
initiation_months_ln = nino34_index.isel(time=times_ln).month

In [22]:
initiation_months_ln

<xarray.DataArray 'month' (time: 13)>
array([ 5,  8,  7,  9,  7, 12,  7, 11,  9,  9,  8, 10, 12])
Coordinates:
  * time        (time) datetime64[ns] 1950-05-01 1954-08-01 ... 2017-12-01
    month       (time) int64 5 8 7 9 7 12 7 11 9 9 8 10 12
    seasonyear  (time) int64 1950 1954 1964 1970 1973 ... 2007 2010 2016 2018

In [23]:
duration_ln = la_nina_events.event_duration.values

In [24]:
duration_ln

array([12, 30,  9, 19, 36, 11, 14,  7, 33, 12, 22,  6,  7])